## Load data into Snowflake tables
To begin, download the full dataset (1 million rows) as a zip from this [Kaggle link](https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning). Then unzip it to a .csv. Load that .csv into your notebook's files directory. We will pull from that csv to create a training data table and our incoming "streamed" data table.

In [ ]:
# Import python packages
import pandas as pd

# Create the session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


Load data from csv into Pandas DataFrame

In [ ]:
# Load full 1M dataset into dataframe
insurance_df = pd.read_csv('insurance_dataset.csv')

Data cleaning, rearranging columns

In [ ]:
insurance_df.head(2)

In [ ]:
# Capitalize column names
insurance_df.columns = insurance_df.columns.str.upper()

# Rearrange columns to fit target schema
cols = insurance_df.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
insurance_df = insurance_df[cols]

In [ ]:
insurance_df.head(1)

Use the write_pandas() method to write the first 10k rows into the 'SOURCE_OF_TRUTH' table created with the SQL commands in the SQL file. The method "returns a Snowpark DataFrame object referring to the table where the pandas DataFrame was written to." (Snowpark Documentation)

In [ ]:
source_of_truth_df = session.write_pandas(insurance_df[:10000], table_name='SOURCE_OF_TRUTH',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

The code below writes the remaining 990k to the INCOMING_DATA_SOURCE table to simulate data being streamed in

In [ ]:
incoming_data_source_df = session.write_pandas(insurance_df[10000:], table_name='INCOMING_DATA_SOURCE',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

## Cars sales - Converting a categorical value into numerical value

In [ ]:
car_sales = pd.read_csv("ML_INSURANCE/car-sales-extended.csv")
car_sales.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
X = car_sales.drop("Price", axis=1)
y = car_sales["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                   y, 
                                                   test_size=0.2,
                                                   random_state=42)


In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
# car_sales.columns.str.upper()
car_sales.columns

In [ ]:
# converting categorical columns into numbers

categorical_columns = ['Make', 'Colour', 'Doors']

one_hot = OneHotEncoder()
transformer = ColumnTransformer(
    [("one_hot_encoder", one_hot, categorical_columns )],
    remainder="passthrough"
    )
transformed_X = transformer.fit_transform(X)
transformed_X

In [ ]:
transformer

In [ ]:
# transformer.get_params()
# transformer.get_feature_names_out()
# transformer.named_transformers_["one_hot_encoder"].get_feature_names_out()
transformer.named_transformers_["one_hot_encoder"].get_feature_names_out(categorical_columns)

In [ ]:
categorical_columns

In [ ]:

import pandas as pd

transformer.fit(X_train)
X_train_transformed = transformer.transform(X_train)

# Feature names
ohe_feature_names = transformer.named_transformers_['one_hot_encoder'] \
    .get_feature_names_out(categorical_columns)

# Pass-through feature names (numerical, etc.)
passthrough_cols = [c for c in X_train.columns if c not in categorical_columns]

all_feature_names = list(ohe_feature_names) + passthrough_cols

X_train_df = pd.DataFrame(X_train_transformed, columns=all_feature_names, index=X_train.index)


In [ ]:
for c in X_train.columns:
    if c not in categorical_columns:
        print(c)

In [ ]:
ohe_feature_names

In [ ]:
X_train_df

In [ ]:
pd.DataFrame(X_train_df)

## Snoeflake Test registery

In [ ]:
select current_database(), current_schema()

In [ ]:
from snowflake.ml.registry import Registry
from sklearn import datasets, ensemble

DATABASE = 'INSURANCE'
SCHEMA = 'ML_PIPE'

registry = Registry(session=session, database_name=DATABASE, schema_name=SCHEMA)

iris_X, iris_y = datasets.load_iris(return_X_y=True, as_frame=True)

# Rename columns so they are valid Snowflake identifiers
column_name_map = {
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width'
}
iris_X = iris_X.rename(columns=column_name_map)

# Train the model
clf = ensemble.RandomForestClassifier(random_state=42)
clf.fit(iris_X, iris_y)

# Log the model in the registry
model_ref = registry.log_model(
    clf,
    model_name="RandomForestClassifier",
    version_name="v1",
    sample_input_data=iris_X,
    options={
        "method_options": {
            "predict": {"case_sensitive": True},
            "predict_proba": {"case_sensitive": True},
            "predict_log_proba": {"case_sensitive": True},
        }
    },
)

# Generate predictions
model_ref.run(iris_X[-10:], function_name='"predict_proba"')

# Pipelines can also be logged in the registry
from sklearn import pipeline, preprocessing

pipe = pipeline.Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('classifier', ensemble.RandomForestClassifier(random_state=42)),
])
pipe.fit(iris_X, iris_y)

model_ref = registry.log_model(
    pipe,
    model_name="Pipeline",
    version_name="v1",
    sample_input_data=iris_X,
    options={
        "method_options": {
            "predict": {"case_sensitive": True},
            "predict_proba": {"case_sensitive": True},
            "predict_log_proba": {"case_sensitive": True},
        }
    },
)

# Generate predictions
model_ref.run(iris_X[-10:], function_name='"predict_proba"')

### Executing above step in chunks

In [ ]:
from snowflake.ml.registry import Registry
from sklearn import datasets, ensemble

DATABASE = 'INSURANCE'
SCHEMA = 'ML_PIPE'

registry = Registry(session=session, database_name=DATABASE, schema_name=SCHEMA)

iris_X, iris_y = datasets.load_iris(return_X_y=True, as_frame=True)

# Rename columns so they are valid Snowflake identifiers
column_name_map = {
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width'
}
iris_X = iris_X.rename(columns=column_name_map)

